<a href="https://colab.research.google.com/github/fmbento/Flowise_AI_chatBots/blob/main/scriptsUA/BC_AV_BibsUA_Extrair_texto_LibGuides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install requests
pip install bs4
touch privGuides.txt

In [ ]:
# Gera URLs_LibGuides.txt do sitemap --
# SE HOUVEREM guias privados devem ser colocadas as URLs no
# ficheiro local privGuides.txt, uma URL do guia por linha -- as URLs devem
# começar por https://uapt.libguides.com/

import requests
from bs4 import BeautifulSoup

sitemap_url = 'https://uapt.libguides.com/sitemap.xml'
base_url = 'https://uapt.libguides.com/'

def crawl_url(url, unique_urls):
    # Get the page content
    page_content = requests.get(url).content

    # Parse the page content
    page_soup = BeautifulSoup(page_content, 'html.parser')

    # Find all the links in the page
    links = page_soup.find_all('a', href=True)

    # Iterate over each link in the page
    for link in links:
        href = link['href']

        # Check if the link starts with the base URL
        if href.startswith(base_url):
            # Add the link to the set of unique URLs
            unique_urls.add(href)

# Get the sitemap content
sitemap_content = requests.get(sitemap_url).content

# Parse the sitemap content
sitemap_soup = BeautifulSoup(sitemap_content, 'xml')

# Get all the URLs from the sitemap
urls = [loc.text for loc in sitemap_soup.find_all('loc')]

# Initialize a set to store the unique URLs
unique_urls = set()

# Iterate over each URL in the sitemap and crawl for links
for url in urls:
    # Add the URL from the sitemap to the set of unique URLs
    unique_urls.add(url)

    crawl_url(url, unique_urls)

# Read additional URLs from privGuides.txt and add them to unique_urls
with open('privGuides.txt', 'r') as f:
    for line in f:
        url = line.strip()
        if url:
            unique_urls.add(url)
            crawl_url(url, unique_urls)

# Write the unique URLs to a file
with open('URLs_LibGuides.txt', 'w') as f:
    for url in sorted(unique_urls):
        f.write(url + '\n')

In [ ]:
# Faz o cranwl de todas as URLS:

!find . -type f -name "*.txt" ! -name "URLs_LibGuides.txt" ! -name "privGuides.txt" -delete

import requests
from bs4 import BeautifulSoup
import re

def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    div = soup.find("div", id="s-lg-guide-main")
    if div:
        text = div.get_text()
    else:
        text = ""
    return text

def generate_filename(index, title):
    title = re.sub(r'[^\w\s-]', '', title).strip()  # Remove special characters
    title = re.sub(r'[-\s]+', '-', title)  # Replace spaces with hyphens
    filename = f"{index:03d}_{title}.txt"
    return filename

def clean_text(text):
    # Remove more than two consecutive empty lines
    text = re.sub(r'\n\s*\n\s*\n\s*\n', '\n\n', text)
    # Remove empty lines that don't have another empty line before or after
    # text = re.sub(r'(?<!\n)\n\s*\n(?!\n)', '\n', text)
    # Remove lines that start with " << Anterior:"
    text = re.sub(r'^\s*<< Anterior:.*\n?', '', text, flags=re.MULTILINE)
    return text

# Read the URLs from the text file
with open("URLs_LibGuides.txt", "r") as file:
    urls = file.read().splitlines()

# Extract text from pages and save in separate files
for index, url in enumerate(urls, start=1):
    try:
        text = extract_text(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.find("title").get_text()

        # Remove the text " - LibGuides at Universidade de Aveiro" from the title
        title = title.replace(" - LibGuides at Universidade de Aveiro", "")

        filename = generate_filename(index, title)

        # Clean the extracted text
        cleaned_text = clean_text(text)

        # Add a blank line before the additional text if necessary
        if not cleaned_text.endswith('\n\n'):
            cleaned_text += '\n'

        # Add the additional text at the end of the file
        cleaned_text += f"\nPara saber mais sobre {title}, aceda a {url}"

        # Remove empty lines at the beginning of the file
        cleaned_text = cleaned_text.lstrip('\n')

        with open(filename, "w", encoding="utf-8") as output_file:
            output_file.write(cleaned_text)

        print(f"Extracted text from page {index}: {filename}")
    except Exception as e:
        print(f"An error occurred while extracting text from page {index}: {e}")

Extracted text from page 1: 001_Início.txt
Extracted text from page 2: 002_PROA-Plataforma-de-Revistas-em-Open-Access-Kit-de-apoio-aos-editores-das-revistas-UA.txt
Extracted text from page 3: 003_Apresentação-Produção-de-conteúdos-digitais-acessíveis.txt
Extracted text from page 4: 004_Bem-vindos-à-Biblioteca-da-ESTGA-UA-Biblioteca-da-ESTGA-UA.txt
Extracted text from page 5: 005_Mendeley-Biblioteca-da-ESTGA-UA.txt
Extracted text from page 6: 006_APA-7ed-Biblioteca-da-ESTGA-UA.txt
Extracted text from page 7: 007_Evitar-o-plágio-Biblioteca-da-ESTGA-UA.txt
Extracted text from page 8: 008_Pesquisar-informação-Biblioteca-da-ESTGA-UA.txt
Extracted text from page 9: 009_Serviços-Biblioteca-da-ESTGA-UA.txt
Extracted text from page 10: 010_Zotero-Biblioteca-da-ESTGA-UA.txt
Extracted text from page 11: 011_Apresentação-Produção-de-conteúdos-digitais-acessíveis.txt
Extracted text from page 12: 012_Documentos-acessíveis-Produção-de-conteúdos-digitais-acessíveis.txt
Extracted text from page 13: 013

In [ ]:
%%bash
zip libguides_txt.zip *.txt -x URLs_LibGuides.txt privGuides.txt
find . -type f -name "*.txt" ! -name "URLs_LibGuides.txt" ! -name "privGuides.txt" -delete

# FEITO!
### basta fazer o download do libguides_txt.zip

o que está abaixo é apenas arquivo das tentativas anteriores

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    div = soup.find("div", id="s-lg-guide-main")
    if div:
        text = div.get_text()
    else:
        text = ""
    return text


def generate_filename(index, title):
    title = re.sub(r'[^\w\s-]', '', title).strip()  # Remove caracteres especiais
    title = re.sub(r'[-\s]+', '-', title)  # Substitui espaços por hífens
    filename = f"{index:03d}_{title}.txt"
    return filename

# Ler as URLs do arquivo de texto
with open("URLs_LibGuides.txt", "r") as file:
    urls = file.read().splitlines()

# Extrair texto das páginas e salvar em arquivos separados
for index, url in enumerate(urls, start=1):
    try:
        text = extract_text(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.find("title").get_text()

        filename = generate_filename(index, title)

        with open(filename, "w", encoding="utf-8") as output_file:
            output_file.write(text)

        print(f"Extraído o texto da página {index}: {filename}")
    except Exception as e:
        print(f"Ocorreu um erro ao extrair o texto da página {index}: {e}")

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    div = soup.find("div", id="s-lg-guide-main")
    if div:
        text = div.get_text()
    else:
        text = ""
    return text

def generate_filename(index, title):
    title = re.sub(r'[^\w\s-]', '', title).strip()  # Remove special characters
    title = re.sub(r'[-\s]+', '-', title)  # Replace spaces with hyphens
    filename = f"{index:03d}_{title}.txt"
    return filename

def clean_text(text):
    # Remove four consecutive empty lines
    text = re.sub(r'\n\s*\n\s*\n\s*\n\s*\n', '\n\n------------------------------------------------\n\n', text)
    # Remove empty lines that don't have another empty line before or after
    text = re.sub(r'(?<!\n)\n\s*\n(?!\n)', '\n', text)
    return text

# Read the URLs from the text file
with open("URLs_LibGuides.txt", "r") as file:
    urls = file.read().splitlines()

# Extract text from pages and save in separate files
for index, url in enumerate(urls, start=1):
    try:
        text = extract_text(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.find("title").get_text()

        filename = generate_filename(index, title)

        # Clean the extracted text
        cleaned_text = clean_text(text)

        with open(filename, "w", encoding="utf-8") as output_file:
            output_file.write(cleaned_text)

        print(f"Extracted text from page {index}: {filename}")
    except Exception as e:
        print(f"An error occurred while extracting text from page {index}: {e}")


In [ ]:
%%capture
!pip install pdfkit wkhtmltopdf

In [ ]:
import pdfkit

def generate_pdf(urls, output_file):
    options = {
        'quiet': '',
        'page-size': 'A4',
        'margin-top': '0mm',
        'margin-right': '0mm',
        'margin-bottom': '0mm',
        'margin-left': '0mm',
        'no-outline': None
    }

    pdfkit.from_file(urls, output_file, options=options)

# Example usage
input_file = 'URLs_LibGuides.txt'
output_pdf = 'output.pdf'

with open(input_file, 'r') as file:
    url_list = file.read().splitlines()

generate_pdf(url_list, output_pdf)
print(f'PDF file "{output_pdf}" generated successfully.')

In [ ]:
import pdfkit
from bs4 import BeautifulSoup
import requests

# Read URLs from file
with open('URLs_LibGuides.txt', 'r') as f:
    urls = f.read().splitlines()

# Fetch web pages and extract content
content = ''
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('title').text.replace(' - LibGuides at Universidade de Aveiro', '')
    main_content = soup.find('div', {'id': 's-lg-guide-main'})
    if main_content:
        content += f'<p></p><h1>{title}</h1><p></p>{main_content}'

# Save content to HTML file
with open('output.html', 'w') as f:
    f.write(content)

# Render HTML and save to PDF
# pdfkit.from_file('output.html', 'output.pdf')


In [ ]:
import pdfkit
from bs4 import BeautifulSoup
import requests

# Read URLs from file
with open('URLs_LibGuides.txt', 'r') as f:
    urls = f.read().splitlines()

# Fetch web pages and extract content
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('title').text.replace(' - LibGuides at Universidade de Aveiro', '')
    main_content = soup.find('div', {'id': 's-lg-guide-main'})
    if main_content:
        content = f'<p></p><h1>{title}</h1><p></p>{main_content}'

        # Save content to HTML file
        filename = f'{title}.html'
        with open(filename, 'w') as f:
            f.write(content)
        print(f'Saved page "{title}" as {filename}')


In [ ]:
from bs4 import BeautifulSoup
import requests

# Read URLs from file
with open('URLs_www_sBIDM.txt', 'r') as f:
    urls = f.read().splitlines()

# Fetch web pages and extract content
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('title').text.replace(' - sbidm - Universidade de Aveiro', '')

    # Find the body tag and extract its text
    body = soup.find('body')
    body_text = body.get_text()

    if body_text:
        content = f'<p></p><h1>{title}</h1><p></p>{body_text}'

        # Save content to HTML file
        filename = f'{title}.html'
        with open(filename, 'w') as f:
            f.write(content)
        print(f'Saved page "{title}" as {filename}')

Saved page "Galerias e espaços - sbidm -  Universidade de Aveiro" as Galerias e espaços - sbidm -  Universidade de Aveiro.html
Saved page "Museu e Arquivo - sbidm -  Universidade de Aveiro" as Museu e Arquivo - sbidm -  Universidade de Aveiro.html
Saved page "UA Editora - sbidm -  Universidade de Aveiro" as UA Editora - sbidm -  Universidade de Aveiro.html
Saved page "visitas - sbidm -  Universidade de Aveiro" as visitas - sbidm -  Universidade de Aveiro.html
Saved page "Conhecer - sbidm -  Universidade de Aveiro" as Conhecer - sbidm -  Universidade de Aveiro.html
Saved page "Utilizar - sbidm -  Universidade de Aveiro" as Utilizar - sbidm -  Universidade de Aveiro.html
Saved page "ajuda e suporte na investigação - sbidm -  Universidade de Aveiro" as ajuda e suporte na investigação - sbidm -  Universidade de Aveiro.html
Saved page "ajuda e suporte - sbidm -  Universidade de Aveiro" as ajuda e suporte - sbidm -  Universidade de Aveiro.html
Saved page "acessibilidade e inclusão - sbidm - 

In [ ]:
%%bash
zip www_sBIDM.zip *.html

updating: acessibilidade e inclusão - sbidm -  Universidade de Aveiro.html (deflated 20%)
updating: ajuda e suporte na investigação - sbidm -  Universidade de Aveiro.html (deflated 13%)
updating: ajuda e suporte - sbidm -  Universidade de Aveiro.html (deflated 17%)
updating: bibliotecas e horários - sbidm -  Universidade de Aveiro.html (deflated 16%)
updating: Conhecer - sbidm -  Universidade de Aveiro.html (deflated 19%)
updating: Galerias e espaços - sbidm -  Universidade de Aveiro.html (deflated 16%)
updating: Gerir informação - sbidm -  Universidade de Aveiro.html (deflated 17%)
updating: Museu e Arquivo - sbidm -  Universidade de Aveiro.html (deflated 17%)
updating: organização - sbidm -  Universidade de Aveiro.html (deflated 18%)
updating: Parcerias e projetos - sbidm -  Universidade de Aveiro.html (deflated 16%)
updating: Serviços de Biblioteca, Informação Documental e Museologia - sbidm -  Universidade de Aveiro.html (deflated 11%)
updating: serviços - sbidm -  Universidade de 

In [ ]:
# Gera URLs_LibGuides.txt do sitemap --
# SE NÃO HOUVER guias privados (ficheiro local privGuides.txt)

import requests
from bs4 import BeautifulSoup

sitemap_url = 'https://uapt.libguides.com/sitemap.xml'
base_url = 'https://uapt.libguides.com/'

# Get the sitemap content
sitemap_content = requests.get(sitemap_url).content

# Parse the sitemap content
sitemap_soup = BeautifulSoup(sitemap_content, 'xml')

# Get all the URLs from the sitemap
urls = [loc.text for loc in sitemap_soup.find_all('loc')]

# Initialize a set to store the unique URLs
unique_urls = set()

# Iterate over each URL in the sitemap
for url in urls:
    # Add the URL from the sitemap to the set of unique URLs
    unique_urls.add(url)

    # Get the page content
    page_content = requests.get(url).content

    # Parse the page content
    page_soup = BeautifulSoup(page_content, 'html.parser')

    # Find all the links in the page
    links = page_soup.find_all('a', href=True)

    # Iterate over each link in the page
    for link in links:
        href = link['href']

        # Check if the link starts with the base URL
        if href.startswith(base_url):
            # Add the link to the set of unique URLs
            unique_urls.add(href)

# Write the unique URLs to a file
with open('URLs_LibGuides.txt', 'w') as f:
    for url in sorted(unique_urls):
        f.write(url + '\n')